In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import nltk
import sklearn
import matplotlib.pyplot as plt
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import *
from keras.regularizers import l1,l2, L1L2

In [2]:
seed_value = 273958
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)

In [3]:
# load in tweets
with open('../data/pickled_tweets/home_2019_3daysback_nolikecriterion.pkl', 'rb') as f:
    home_2019_tweets = pickle.load(f)
with open('../data/pickled_tweets/away_2019_3daysback_nolikecriterion.pkl', 'rb') as f:
    away_2019_tweets = pickle.load(f)
with open('../data/pickled_tweets/home_2020_3daysback_nolikecriterion.pkl', 'rb') as f:
    home_2020_tweets = pickle.load(f)
with open('../data/pickled_tweets/away_2020_3daysback_nolikecriterion.pkl', 'rb') as f:
    away_2020_tweets = pickle.load(f)

# load in nfl data
s2020 = pd.read_csv('../data/season_data/2020_all_data.csv', index_col=0)
s2019 = pd.read_csv('../data/season_data/2019_all_data.csv', index_col=0)

In [4]:
sw = stopwords.words('english')

# lemmatize function
def lemmatize(sentence, include_stopwords=False):
    if include_stopwords:
        return [WordNetLemmatizer().lemmatize(word) for word in sentence]
    return [WordNetLemmatizer().lemmatize(word) for word in sentence if word not in sw]

# preprocess the tweets - remove punctuation and lemmatize
def preprocess(tweets):
    for i in range(len(tweets)):
        for j in range(len(tweets[i])):
            tweets[i][j] = re.sub('[^a-zA-Z]',' ',tweets[i][j]).split()
            tweets[i][j] = lemmatize(tweets[i][j])

preprocess(home_2019_tweets)
preprocess(away_2019_tweets)
preprocess(home_2020_tweets)
preprocess(away_2020_tweets)

In [5]:
from collections import Counter
def vectorize_list(list_of_tweets, corpus):
    num_tweets = len(list_of_tweets)
    lol = [nltk.bigrams(x) for x in list_of_tweets]
    with_repeats = [item for sublist in lol for item in sublist]
    counts = dict(Counter(with_repeats))
    to_return = []
    for key in corpus:
        num = np.log(1+counts[key]) if key in counts else 0
        to_return.append(num)
    return to_return

In [6]:
# count bigrams
def count_bigrams(tweet,corpus):
    bigramized_tweet = nltk.bigrams(tweet)
    for bigram in bigramized_tweet:
        if bigram in corpus:
            corpus[bigram] += 1
        else:
            corpus[bigram] = 1

In [7]:
#Create the home/away corpus
home_corpus = {}
away_corpus = {}
for tweets in home_2019_tweets:
    for tw in tweets:
        count_bigrams(tw,home_corpus)

for tweets in away_2019_tweets:
    for tw in tweets:
        count_bigrams(tw,away_corpus)

In [8]:
stopwords = stopwords.words('english')

In [9]:
# remove words with <= 2 characters, stopwords, hashtags
for key in list(home_corpus.keys()):
    if key[0] == 'http' or key[1] == 'http':
        del home_corpus[key]

for key in list(away_corpus.keys()):
    if key[0] == 'http' or key[1] == 'http':
        del away_corpus[key]

In [10]:
# get number of tweets for all home and away teams
n_home_tweets = sum([len(game_tweets) for game_tweets in home_2019_tweets])
n_away_tweets = sum([len(game_tweets) for game_tweets in away_2019_tweets])

# get the unigrams that appear in at least 0.1% of home/away tweets
home_top_grams = [word for word in home_corpus if home_corpus[word] > n_home_tweets*0.0001]
away_top_grams = [word for word in away_corpus if away_corpus[word] > n_away_tweets*0.0001]

In [11]:
# TRAIN SET
#Create lists of vectors for home, away games
num_h_tweets = [vectorize_list(game, home_top_grams) for game in home_2019_tweets]
num_a_tweets = [vectorize_list(game, away_top_grams) for game in away_2019_tweets]

#Turn into arrays
home_vecs = np.array(num_h_tweets)
away_vecs = np.array(num_a_tweets)

#Concatenate home, away to form input matrix.
X_train = np.concatenate([home_vecs, away_vecs], axis=1)

# TEST SET
#Create lists of vectors for home, away games
num_h_tweets = [vectorize_list(game, home_top_grams) for game in home_2020_tweets]
num_a_tweets = [vectorize_list(game, away_top_grams) for game in away_2020_tweets]

#Turn into arrays
home_vecs = np.array(num_h_tweets)
away_vecs = np.array(num_a_tweets)

#Concatenate home, away to form input matrix.
X_test = np.concatenate([home_vecs, away_vecs], axis=1)

# TRAIN and TEST outcomes
Y_train = np.array(s2019["Home Win"])
Y_test = np.array(s2020["Home Win"])

In [12]:
# Basic classifiers on full data
lr = LogisticRegression(penalty="l1",solver="liblinear",C=0.1)
lr.fit(X_train, Y_train)
print(lr.score(X_test, Y_test))

lrl1 = LogisticRegressionCV(cv=5,penalty="l1",solver="liblinear",
                            max_iter=1000).fit(X_train, Y_train)
lrl1.fit(X_train, Y_train)
print(lrl1.score(X_test, Y_test))

lrl2 = LogisticRegressionCV(cv=5,penalty="l2",solver="liblinear",
                            max_iter=1000).fit(X_train, Y_train)
lrl2.fit(X_train, Y_train)
print(lrl2.score(X_test, Y_test))

ada = AdaBoostClassifier(n_estimators=200)
ada.fit(X_train, Y_train)
print(ada.score(X_test, Y_test))

rf = RandomForestClassifier(n_estimators=400)
rf.fit(X_train, Y_train)
print(rf.score(X_test, Y_test))

gnb = GaussianNB().fit(X_train, Y_train)
gnb.fit(X_train, Y_train)
print(gnb.score(X_test, Y_test))

0.56640625
0.61328125
0.6015625
0.5546875
0.6015625
0.5234375


### Sparse L1 Coefficients

In [13]:
len(np.where(lr.coef_ != 0)[1]) #sparsity of lr coefs

26

In [14]:
all_grams = np.array(home_top_grams + away_top_grams)
all_grams[np.where(lr.coef_ != 0)[1]] #grams corresponding to nonzero entries

array([['first', 'time'],
       ['thursday', 'night'],
       ['super', 'bowl'],
       ['con', 'el'],
       ['wait', 'see'],
       ['co', 'ryv'],
       ['pro', 'bowl'],
       ['josh', 'gordon'],
       ['see', 'soon'],
       ['ravensflock', 'ravensnation'],
       ['injury', 'report'],
       ['de', 'los'],
       ['get', 'ready'],
       ['key', 'matchup'],
       ['next', 'year'],
       ['kansa', 'city'],
       ['practice', 'today'],
       ['senior', 'night'],
       ['yard', 'td'],
       ['h', 'r'],
       ['tampa', 'bay'],
       ['chief', 'chiefskingdom'],
       ['ravensflock', 'ravensnation'],
       ['bengal', 'whodey'],
       ['russell', 'wilson'],
       ['chase', 'young']], dtype='<U24')

In [15]:
np.where(lr.coef_ != 0)[1]

array([   33,    39,   103,   211,   321,   568,  1431,  2380,  3386,
        6265,  7706,  8101,  8161,  8465,  8675,  8845,  8939,  9195,
        9847, 10051, 11312, 11515, 11770, 11894, 11896, 12129])

In [16]:
signs = np.sign(np.take(lr.coef_, np.where(lr.coef_ != 0)[1])).reshape(-1,1)

In [17]:
home_away = np.where(np.where(lr.coef_ != 0)[1] > len(home_top_grams), 'A', 'H').reshape(-1,1)

In [18]:
np.concatenate([all_grams[np.where(lr.coef_ != 0)[1]], signs, np.where(lr.coef_ != 0)[1].reshape(-1,1), home_away], axis=1)

array([['first', 'time', '1.0', '33', 'H'],
       ['thursday', 'night', '1.0', '39', 'H'],
       ['super', 'bowl', '1.0', '103', 'H'],
       ['con', 'el', '1.0', '211', 'H'],
       ['wait', 'see', '1.0', '321', 'H'],
       ['co', 'ryv', '1.0', '568', 'H'],
       ['pro', 'bowl', '1.0', '1431', 'H'],
       ['josh', 'gordon', '1.0', '2380', 'H'],
       ['see', 'soon', '1.0', '3386', 'H'],
       ['ravensflock', 'ravensnation', '1.0', '6265', 'H'],
       ['injury', 'report', '-1.0', '7706', 'A'],
       ['de', 'los', '-1.0', '8101', 'A'],
       ['get', 'ready', '-1.0', '8161', 'A'],
       ['key', 'matchup', '-1.0', '8465', 'A'],
       ['next', 'year', '1.0', '8675', 'A'],
       ['kansa', 'city', '-1.0', '8845', 'A'],
       ['practice', 'today', '-1.0', '8939', 'A'],
       ['senior', 'night', '1.0', '9195', 'A'],
       ['yard', 'td', '-1.0', '9847', 'A'],
       ['h', 'r', '-1.0', '10051', 'A'],
       ['tampa', 'bay', '-1.0', '11312', 'A'],
       ['chief', 'chiefskingdom',

### Cross Validated L1

In [19]:
len(np.where(lrl1.coef_ != 0)[1]) #sparsity of lr coefs

4539

In [20]:
max_positives = np.argpartition(lrl1.coef_.reshape(-1), -20)[-20:]
all_grams[max_positives]

array([['win', 'st'],
       ['knee', 'full'],
       ['second', 'game'],
       ['para', 'que'],
       ['dfs', 'draftkings'],
       ['claim', 'josh'],
       ['rts', 'appreciated'],
       ['kenny', 'moore'],
       ['th', 'place'],
       ['paying', 'attention'],
       ['w', 'l'],
       ['sport', 'falcon'],
       ['former', 'pro'],
       ['sex', 'bestselling'],
       ['qb', 'league'],
       ['free', 'throw'],
       ['er', 'raven'],
       ['david', 'njoku'],
       ['team', 'ever'],
       ['pay', 'attention']], dtype='<U24')

In [21]:
max_negatives = np.argpartition(-1*lrl1.coef_.reshape(-1), -20)[-20:]
all_grams[max_negatives]

array([['get', 'go'],
       ['financial', 'insight'],
       ['fan', 'wanted'],
       ['whitney', 'mercilus'],
       ['lesean', 'mccoy'],
       ['ticket', 'promo'],
       ['team', 'even'],
       ['jjy', 'g'],
       ['bengal', 'seizethedey'],
       ['well', 'others'],
       ['h', 'h'],
       ['lot', 'fun'],
       ['player', 'would'],
       ['co', 'ao'],
       ['time', 'come'],
       ['fantasyfootball', 'week'],
       ['excited', 'share'],
       ['need', 'show'],
       ['addition', 'etsy'],
       ['co', 'um']], dtype='<U24')

### Cross Validated L2

In [22]:
len(np.where(lrl2.coef_ != 0)[1]) #sparsity of lr coefs

15315

In [23]:
max_positives = np.argpartition(lrl2.coef_.reshape(-1), -20)[-20:]
all_grams = np.array(home_top_grams + away_top_grams)
all_grams[max_positives]

array([['mason', 'rudolph'],
       ['bengal', 'whodey'],
       ['thursday', 'night'],
       ['first', 'time'],
       ['chase', 'young'],
       ['lo', 'que'],
       ['de', 'la'],
       ['josh', 'gordon'],
       ['super', 'bowl'],
       ['en', 'el'],
       ['see', 'soon'],
       ['f', 'r'],
       ['wait', 'see'],
       ['wr', 'josh'],
       ['n', 'de'],
       ['con', 'el'],
       ['pro', 'bowl'],
       ['de', 'los'],
       ['ryv', 'edmr'],
       ['co', 'ryv']], dtype='<U24')

In [24]:
signs = np.sign(np.take(lrl2.coef_, max_positives)).reshape(-1,1)
home_away = np.where(max_positives > len(home_top_grams), 'A', 'H').reshape(-1,1)
np.concatenate([all_grams[max_positives], signs, max_positives.reshape(-1,1), home_away], axis=1)

array([['mason', 'rudolph', '1.0', '6972', 'H'],
       ['bengal', 'whodey', '1.0', '11894', 'A'],
       ['thursday', 'night', '1.0', '39', 'H'],
       ['first', 'time', '1.0', '33', 'H'],
       ['chase', 'young', '1.0', '12129', 'A'],
       ['lo', 'que', '1.0', '1809', 'H'],
       ['de', 'la', '1.0', '985', 'H'],
       ['josh', 'gordon', '1.0', '2380', 'H'],
       ['super', 'bowl', '1.0', '103', 'H'],
       ['en', 'el', '1.0', '87', 'H'],
       ['see', 'soon', '1.0', '3386', 'H'],
       ['f', 'r', '1.0', '1947', 'H'],
       ['wait', 'see', '1.0', '321', 'H'],
       ['wr', 'josh', '1.0', '2090', 'H'],
       ['n', 'de', '1.0', '213', 'H'],
       ['con', 'el', '1.0', '211', 'H'],
       ['pro', 'bowl', '1.0', '1431', 'H'],
       ['de', 'los', '1.0', '208', 'H'],
       ['ryv', 'edmr', '1.0', '569', 'H'],
       ['co', 'ryv', '1.0', '568', 'H']], dtype='<U32')

In [25]:
max_negatives = np.argpartition(-1*lrl2.coef_.reshape(-1), -20)[-20:]
all_grams = np.array(home_top_grams + away_top_grams)
all_grams[max_negatives]

array([['er', 'goniners'],
       ['tampa', 'bay'],
       ['redskin', 'httr'],
       ['player', 'week'],
       ['saint', 'fan'],
       ['good', 'thing'],
       ['gohawks', 'seahawks'],
       ['en', 'el'],
       ['super', 'bowl'],
       ['key', 'matchup'],
       ['get', 'ready'],
       ['yard', 'td'],
       ['injury', 'report'],
       ['practice', 'today'],
       ['kansa', 'city'],
       ['de', 'la'],
       ['h', 'r'],
       ['chief', 'chiefskingdom'],
       ['russell', 'wilson'],
       ['de', 'los']], dtype='<U24')

In [26]:
signs = np.sign(np.take(lrl2.coef_, max_negatives)).reshape(-1,1)
home_away = np.where(max_negatives > len(home_top_grams), 'A', 'H').reshape(-1,1)
np.concatenate([all_grams[max_negatives], signs, max_negatives.reshape(-1,1), home_away], axis=1)

array([['er', 'goniners', '-1.0', '12494', 'A'],
       ['tampa', 'bay', '-1.0', '11312', 'A'],
       ['redskin', 'httr', '-1.0', '3587', 'H'],
       ['player', 'week', '-1.0', '10722', 'A'],
       ['saint', 'fan', '-1.0', '10124', 'A'],
       ['good', 'thing', '-1.0', '8336', 'A'],
       ['gohawks', 'seahawks', '-1.0', '13483', 'A'],
       ['en', 'el', '-1.0', '8300', 'A'],
       ['super', 'bowl', '-1.0', '7681', 'A'],
       ['key', 'matchup', '-1.0', '8465', 'A'],
       ['get', 'ready', '-1.0', '8161', 'A'],
       ['yard', 'td', '-1.0', '9847', 'A'],
       ['injury', 'report', '-1.0', '7706', 'A'],
       ['practice', 'today', '-1.0', '8939', 'A'],
       ['kansa', 'city', '-1.0', '8845', 'A'],
       ['de', 'la', '-1.0', '8121', 'A'],
       ['h', 'r', '-1.0', '10051', 'A'],
       ['chief', 'chiefskingdom', '-1.0', '11515', 'A'],
       ['russell', 'wilson', '-1.0', '11896', 'A'],
       ['de', 'los', '-1.0', '8101', 'A']], dtype='<U32')